In [ ]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

import re # to access database details in a file
import time # to time how long queries and processes take
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta # to manipulate dates

from connect_db import connect_db
from makeWeekList import makeWeekList

% matplotlib inline

In [ ]:
path='/Users/brianna/Documents/WL_DBdeets/'

In [ ]:
# Access the 'events' database.
conE, curE = connect_db('evnt', path)

# Access the 'transactions' database.
conT, curT = connect_db('trns', path)

## Query the events database to find out which users visited the site on which weeks

In [ ]:
# List tables in the Transactional (Production) database 'dd8dik067j3tun'

# Retention analysis: cohort retention analysis, look for sql examples online
# Define a cohort as people who did a certain action or behavior
# 90%  of people that signed up viewed internal page in the first week
# Look at weekly retention (how many people viewed an internal page in the following weeks after signing up)
# 30% is good for growing company, facebook has 50%, WL wants at least 20%


In [ ]:
# Make a dict with lists of unique users that visited the site in each particular week.
def listUniqueUsersByWeek(weekList):
    UniqueUserByWeek = {}

    start_time = time.time()

    # Go through all of the weeks except for the last one since it's not finished so you'll get an error.
    for w in range(len(weekList)):
        date_start=str(weekList[w]).split(' ')[0]
        date_end = weekList[w] + timedelta(days=7)
        date_end=str(date_end).split(' ')[0]

        string1="""
            select 
                distinct user_id
            from
                app131567.viewed_meeting_details_page
            where 
                event_time between 
        """
        string2="'{date_start}' and '{date_end}'".format(date_start=date_start, date_end=date_end) 

        sql_query = string1 + string2

        query_result = pd.read_sql_query(sql_query,conE)

        # Add an entry to the dict where the key is the week and the value is the 
        # list of users that accessed the site that week
        UniqueUserByWeek[date_start] = query_result['user_id'].tolist()
        #print(sql_query)

    total_time =  time.time() - start_time
    print('Total time for queries: '+str(total_time))
    
    return UniqueUserByWeek

In [ ]:
def cleanRetention(retention):
    # Delete columns we don't need
    #del retention['reg_date']
    #del retention['Unnamed: 0']
    
    retention = retention[['user_id','reg_week']]

    # Make a variable called registrationWeek which has the week the user registered in the form of a string
    retention['registrationWeek']=retention.reg_week.apply(lambda x : str(x).split(' ')[0])

    # Ay ay ay, get rid of that weird '2016-01-26 10:19:51.044020' entry.  Not sure how it got there, 
    # but it's stamped from this morning.
    #retention = retention[retention.reg_week != '2016-01-26 10:19:51.044020']
    #retention = retention[retention.reg_week != '2016-01-28 20:49:31.258509']
    retention = retention[retention.reg_week.notnull()]
    #print(retention.head(5))
    return retention

In [ ]:
# Go 8 weeks out from registration week (don't worry about anything after that.)

# Query from  2015-05-31, fill it in as Week 0 for people who registered that week.
# Query from  2015-06-07, fill it in as Week 0 for people who registered that week 
#                                   and Week 1 for people that registered the week before.
# Etc.

# This is kind of clunky, but I need to re-define the function each time we go to a new 
# week since the value of x (the number of weeks since registration) changes in each
# iteration but I can't pass that dumb argument through the apply function.
def checkVisitedByWeek(retention, UniqueUserByWeek):
    for week in weekStringList:

        print(week)
        def checkVisited(retention):
            if retention['user_id'] in UniqueUserByWeek[week]:
                return 1
            else:
                return 0

        retention['Visited'+week.replace('-','_')]=retention.apply(lambda x : checkVisited(x), axis=1)

    #retention.head(10)
    return retention

In [ ]:
# Make a dataframe with an entry for user_id and registrationWeek, then a variable for
# whether the user visited in each of the 8 weeks following registration.
# Later, we're going to add a variable to track whether this user visited on this week.
#dtype={'user_id':str,'reg_date':datetime.datetime, 'reg_week':datetime.datetime}

retention = pd.read_csv(path+'03_UserRegistrationDateMoreDetails.csv', low_memory=False)

retention = cleanRetention(retention)

# First make a list of all of the weeks we're interested in.
start_date = '2015-05-24'; end_date = '2016-01-26'
weekList, weekStringList = makeWeekList(start_date, end_date)

# Sometimes there's an error if there are funky values in here.
#retention.reg_week.unique()

# Make a dict where the key is the week and the value is the list of unique users that week
UniqueUserByWeek = listUniqueUsersByWeek(weekList)

# Add a column for each week where 1 indicates the user visited the site that week and 0 
# indicates they didn't.
retention = checkVisitedByWeek(retention, UniqueUserByWeek)

In [ ]:
retention.head()

In [ ]:
retentionByCohort = retention.groupby('registrationWeek').sum()

print(retentionByCohort.loc['2015-05-24','Visited2015_05_31'])

In [ ]:
for rw in range(len(weekStringList)):
    for vw in range(len(weekStringList)):
        if vw < rw:
            #retentionByCohort.loc['2015-05-24','Visited2015_05_31']
            retentionByCohort.loc[weekStringList[rw],'Visited'+weekStringList[vw].replace('-','_')]=None
            retention.loc[retention.registrationWeek == weekStringList[rw],'Visited'+weekStringList[vw].replace('-','_')]=None

In [ ]:
retention.head(10)

In [ ]:
retentionByCohort.to_csv(path+'04b_UserRegistrationDateForCohortPlot.csv')
retention.to_csv(path+'04b_UserRegistrationDateForCohortPlotAllUsers.csv')